# Inverse Search

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm

plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [6]:
generator = 'DK onwind'
output_feature = 'LCOE'
below = 162.
significance = 0.6

In [7]:
carrier = generator[3:]
bus = generator[:2]

In [8]:
### identify period-model combinations where outpute_feature leq below
lcoe_ichec = pd.read_csv('data/lceo_per_carrier_per_node_all_periods_model_ICHEC.csv', index_col=[0,1], header=0)
lcoe_cnrm = pd.read_csv('data/lceo_per_carrier_per_node_all_periods_model_CNRM.csv', index_col=[0,1], header=0)
lcoe_mpi = pd.read_csv('data/lceo_per_carrier_per_node_all_periods_model_MPI.csv', index_col=[0,1], header=0)

lcoe_ichec.columns = ['ICHEC-{}'.format(period) for period in lcoe_ichec.columns]
lcoe_mpi.columns = ['MPI-{}'.format(period) for period in lcoe_mpi.columns]
lcoe_cnrm.columns = ['CNRM-{}'.format(period) for period in lcoe_cnrm.columns]

lcoe = pd.concat([lcoe_ichec.loc[(bus, carrier),:], lcoe_mpi.loc[(bus, carrier),:], lcoe_cnrm.loc[(bus, carrier),:]])

model_period_idx = lcoe.index[lcoe <= below]

In [11]:
### investigate average capacity factors
avgcf = pd.read_csv('/home/users/train061/average_capacity_factors.csv', header=0, index_col=0)

avgcf_mn = avgcf.mean()

sample = avgcf.loc[model_period_idx,:]

avgcf_sample_mn = sample.mean()

gens_below = avgcf_mn.index[avgcf_sample_mn < avgcf_mn]
gens_above = avgcf_mn.index[avgcf_sample_mn > avgcf_mn]

prob = pd.Series(dtype=float)
for gb in gens_below.append(gens_above):
    if gb in gens_below:
        prob[gb] = np.sum(sample[gb] <= avgcf_mn[gb]) / len(sample.index)
    elif gb in gens_above:
        prob[gb] = np.sum(sample[gb] >= avgcf_mn[gb]) / len(sample.index)

prob = prob.sort_values()
    
gens_below_sig = prob[gens_below].index[prob[gens_below]>=significance]
gens_above_sig = prob[gens_above].index[prob[gens_above]>=significance]

print('When the lcoe of {} is below {}, the average capacity factor of {} is below average and above average for {} (significance of {})'.format(generator, below, gens_below_sig, gens_above_sig, significance))

When the lcoe of DK onwind is below 162.0, the average capacity factor of Index(['BA ror', 'BG onwind', 'BG offwind', 'DK solar', 'EE solar', 'FI solar',
       'GR offwind', 'HR offwind', 'HR ror', 'IT offwind', 'IT ror',
       'LT solar', 'LU solar', 'LV solar', 'PL solar', 'SE solar', 'SI ror'],
      dtype='object') is below average and above average for Index(['AT onwind', 'BE onwind', 'BE offwind', 'CZ onwind', 'CZ ror',
       'DE onwind', 'DE offwind', 'DK onwind', 'DK offwind', 'DK ror',
       'EE onwind', 'EE offwind', 'EE ror', 'FI onwind', 'FI offwind',
       'FI ror', 'GB onwind', 'GB offwind', 'HU onwind', 'IE onwind',
       'IE offwind', 'LT onwind', 'LT offwind', 'LU onwind', 'LV onwind',
       'LV offwind', 'LV ror', 'NL onwind', 'NL offwind', 'NO onwind',
       'NO offwind', 'NO ror', 'PL onwind', 'PL offwind', 'PL ror',
       'SE onwind', 'SE offwind'],
      dtype='object') (significance of 0.6)


In [67]:
### investigate seasonal complementarity
seas_comp = pd.read_csv('/home/users/train061/seasonal_complementarity_order4.csv', header=0, index_col=0)

seas_comp_mn = seas_comp.mean()

sample = seas_comp.loc[model_period_idx,:]

seas_comp_sample_mn = sample.mean()

gens_above = seas_comp_mn.index[seas_comp_sample_mn > seas_comp_mn]

prob = pd.Series(dtype=float)
for gb in gens_above:
    prob[gb] = np.sum(sample[gb] >= seas_comp_mn[gb]) / len(sample.index)

prob = prob.sort_values()
    
gens_above_sig = prob.index[prob>=significance]

print('When the lcoe of {} is below {}, the seasonal complementarity in {} is above average (significance of {})'.format(generator, below, gens_below_sig, significance))

When the lcoe of DK onwind is below 162.0, the seasonal complementarity in Index(['SE ror', 'SE solar', 'SE offwind', 'SE onwind', 'DK solar',
       'LU onwind', 'LU solar', 'LU ror', 'DK ror', 'NL onwind', 'NL offwind',
       'NL solar', 'NL ror', 'DK offwind', 'DK onwind'],
      dtype='object') is above average (significance of 0.6)
